## ●Notebookの内容

変数選択後データ(コード4の「回帰分析による変数選択」で作成したもの)の読み込み、前処理(ランク正規化)、形状変換

# 1. パッケージのインポート・乱数固定

In [1]:
import pandas as pd #pandasパッケージをインポート
import numpy as np #numpyパッケージをインポート
import torch #ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn #「ニューラルネットワーク」モジュールの別名定義
import torch.nn.functional as F #「ニューラルネットワーク・活性化関数」モジュールの別名定義
import collections
import os
import pickle
import optuna
import torch.optim as optim

#乱数の固定
torch.manual_seed(123)

# 2. 整理後データの読み込み

In [2]:
pickle_in = open("./organized_data3.pickle","rb") 
data_4_0 = pickle.load(pickle_in)
pickle_in.close()
display(data_4_0)

,銘柄名,期日,銘柄コード,調整済始値,調整済高値,調整済安値,株価(終値)移動平均25日乖離率,株価(終値)移動平均75日乖離率,株価(終値)移動平均200日乖離率,PER(普通株式数･日経予想･連結優先･含負),...,総還元性向,総資産連単倍率,売上高連単倍率,営業利益連単倍率,経常利益連単倍率,当期利益連単倍率,期末EV/EBITDA,時価総額(発行済株式数ﾍﾞｰｽ),計算用現値（調整済、市場引継）,対TOPIX β値(60ヵ月) 2乗
0,極洋,1994/12,1301.0,3780.0,4190.0,3710.0,2.9185,-1.3674,-7.4138,60.719152,...,34.644268,1.057022,1.049759,1.141331,1.049765,0.946636,35.878976,4.553936e+10,4020.0,1.640167
1,極洋,1995/01,1301.0,4070.0,4130.0,3580.0,-2.8197,-3.9003,-10.0966,58.302469,...,34.801282,1.057022,1.049759,1.141445,1.049182,0.946636,35.878976,4.372685e+10,3860.0,1.621369
2,極洋,1995/02,1301.0,3880.0,4000.0,3460.0,-8.3524,-10.2346,-16.9429,53.167019,...,34.801282,1.057022,1.049759,1.141100,1.047150,0.946636,35.878976,3.987526e+10,3520.0,1.728384
3,極洋,1995/03,1301.0,3520.0,3800.0,3350.0,-3.1339,-10.3753,-17.6057,51.354507,...,34.424683,1.057022,1.049759,1.141879,1.049182,0.946636,35.878976,3.851588e+10,3400.0,1.583596
4,極洋,1995/04,1301.0,3300.0,3700.0,3150.0,1.7775,-3.0125,-11.4933,53.620147,...,34.424683,1.057022,1.049759,1.141879,1.049406,0.946636,35.878976,4.021511e+10,3550.0,1.579018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997.0,883.0,900.0,773.0,-2.3335,-14.7397,-21.8114,6.358036,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,7.993457e+10,822.0,2.385110
370821,ベルーナ,2021/09,9997.0,822.0,892.0,819.0,0.2582,-4.7101,-18.3318,6.605550,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,8.304638e+10,854.0,2.379800
370822,ベルーナ,2021/10,9997.0,848.0,851.0,792.0,-3.3851,-6.3116,-22.2559,6.172400,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,7.760071e+10,798.0,2.348832
370823,ベルーナ,2021/11,9997.0,793.0,795.0,665.0,-10.6664,-17.3400,-32.4331,5.159130,...,14.452700,1.123884,1.598314,1.584172,1.541385,1.483134,8.300202,6.486175e+10,667.0,2.758423


In [3]:
day_num_4 = data_4_0.期日.nunique(dropna = True) #期日の数(月数)
com_num_4 = data_4_0.shape[0]/day_num_4 #上場廃止や途中上場がない使用可能な企業数
#com_num_4 = data_4.銘柄コード.nunique(dropna = True) #上場廃止や途中上場がない使用可能な企業数
print('使用可能な企業数は', com_num_4,'、', '期日数は', day_num_4 ,'である。')

使用可能な企業数は 1141.0 、 期日数は 325 である。


# 3. 変数選択

## 3.1 変数選択後データの読み込み

In [4]:
chosen_data_1 = pd.read_excel('./chosen character.xlsx', header=None, index_col=None)
chosen_data_2 = chosen_data_1.iloc[1:,1:].reset_index().iloc[:,1]
chosen_data_2

0        EPS(普通株式数･日経予想･連結優先)
1                      HV（1年）
2     PER(普通株式数･日経予想･連結優先･含負)
3            他人資本ｺｽﾄ(有利子負債金利)
4                出来高回転率(250日)
5                     売上債権回転率
6                 売上債権対買入債務比率
7                      売上高伸び率
8                      売上高原価率
9                   売上高売上総利益率
10                売上高支払利息割引料率
11                  売上高減価償却費率
12                   売上高経常利益率
13                    売上高連単倍率
14            対TOPIX α値(60ヵ月)
15            対TOPIX β値(60ヵ月)
16                       当座比率
17                    当期利益伸び率
18                   当期利益連単倍率
19                 手元流動性比率(月)
20           時価総額(発行済株式数ﾍﾞｰｽ)
21                  有形固定資産回転率
22                   有形固定資産比率
23           株価(終値)移動平均75日乖離率
24                    流動資産回転率
25                    海外売上高比率
26                         税率
27                     総資産利益率
28                      総還元性向
29               自己資本利益率(ROE)
30                    自己資本配当率
31                     自己金融比率
32                      調整済高値
33        

## 3.2 変数選択の実行

In [5]:
data_4 = pd.concat([data_4_0['銘柄名'], data_4_0['期日'], data_4_0['銘柄コード'], data_4_0['配当込み収益率(月次)'], data_4_0[chosen_data_2]], axis=1)

# 3. 前処理(ランク正規化(-1,1))と書き出し

## 3.1 ランク正規化(0,1)の実行

In [6]:
data_4_1 = data_4.sort_values(by=['期日'])
data_4_1 = data_4_1.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除
data_4_6 = pd.DataFrame()
for e in range(0,day_num_4):
    data_4_2 = data_4_1.loc[0+(com_num_4*e):com_num_4-1+(com_num_4*e),'配当込み収益率(月次)':]
    data_4_3 = data_4_1.loc[0+(com_num_4*e):com_num_4-1+(com_num_4*e),:'銘柄コード']
    data_4_4 = data_4_2.rank(pct = True)
    data_4_5 = pd.concat([data_4_3,data_4_4],axis=1)
    data_4_6 = pd.concat([data_4_6,data_4_5],axis=0)
display(data_4_6)
data_4_7_0 = data_4_6.sort_values(by=['銘柄コード', '期日']) #ランク正規化したものを期日と銘柄コードをキーに並べ替え
data_4_7_0 = data_4_7_0.reset_index(drop=True) #indexの振り直し(元のindexを新しい列にしてそれを削除
display(data_4_7_0) #期日と銘柄コードをキーに並べ替えしたものを表示

,銘柄名,期日,銘柄コード,配当込み収益率(月次),EPS(普通株式数･日経予想･連結優先),HV（1年）,PER(普通株式数･日経予想･連結優先･含負),他人資本ｺｽﾄ(有利子負債金利),出来高回転率(250日),売上債権回転率,...,自己資本配当率,自己金融比率,調整済高値,負債比率,買入債務回転率,過去1年ﾘﾀｰﾝ,配当利回り予想値,長短借入金比率,ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,ﾎﾞﾘｭｰﾑﾚｼｵ
0,極洋,1994/12,1301.0,0.809816,0.225241,0.745837,0.696757,0.387379,0.801052,0.826468,...,0.500438,0.915863,0.784400,0.892200,0.803681,0.367222,0.044698,0.953550,0.116564,0.863278
1,日精樹脂,1994/12,6293.0,0.243646,0.441718,0.295355,0.799299,0.968449,0.107800,0.107800,...,0.035057,0.350570,0.463190,0.241017,0.028922,0.840491,0.300175,0.120070,0.325153,0.988606
2,ウシオ電,1994/12,6925.0,0.939527,0.570552,0.431201,0.552147,0.787905,0.992989,0.292726,...,0.801928,0.639790,0.353637,0.238387,0.279579,0.873795,0.443471,0.368975,0.677476,0.990359
3,クニミネ,1994/12,5388.0,0.401402,0.584575,0.964943,0.639790,0.783523,0.120070,0.134093,...,0.934268,0.732691,0.481595,0.097283,0.674847,0.786152,0.448291,0.180543,0.885188,0.901840
4,第一実,1994/12,8059.0,0.340929,0.244522,0.826468,0.815074,0.422436,0.716915,0.078878,...,0.825592,0.950920,0.720859,0.738826,0.007888,0.909728,0.734882,0.713409,0.276950,0.696757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ＣＫＤ,2021/12,6407.0,0.531113,0.623138,0.858896,0.588957,0.811569,0.820333,0.371604,...,0.425066,0.849255,0.671341,0.297984,0.318142,0.531113,0.390885,0.379492,0.669588,0.868536
370821,太陽誘電,2021/12,6976.0,0.581069,0.903593,0.870289,0.692375,0.168273,0.933392,0.460999,...,0.640666,0.382997,0.943909,0.361087,0.680982,0.858019,0.166521,0.609115,0.952673,0.439965
370822,栃木銀,2021/12,8550.0,0.990359,0.132340,0.555653,0.209465,0.500438,0.971078,0.500438,...,0.007011,0.500438,0.021911,0.982472,0.500438,0.856266,0.393514,0.500438,0.603856,0.992989
370823,帝人,2021/12,3401.0,0.765995,0.659071,0.132340,0.197195,0.345311,0.814198,0.617003,...,0.692375,0.313760,0.408852,0.743208,0.645048,0.057844,0.845749,0.853637,0.741455,0.747590


,銘柄名,期日,銘柄コード,配当込み収益率(月次),EPS(普通株式数･日経予想･連結優先),HV（1年）,PER(普通株式数･日経予想･連結優先･含負),他人資本ｺｽﾄ(有利子負債金利),出来高回転率(250日),売上債権回転率,...,自己資本配当率,自己金融比率,調整済高値,負債比率,買入債務回転率,過去1年ﾘﾀｰﾝ,配当利回り予想値,長短借入金比率,ｷｬｯｼｭﾌﾛｰﾏｰｼﾞﾝ,ﾎﾞﾘｭｰﾑﾚｼｵ
0,極洋,1994/12,1301.0,0.809816,0.225241,0.745837,0.696757,0.387379,0.801052,0.826468,...,0.500438,0.915863,0.784400,0.89220,0.803681,0.367222,0.044698,0.953550,0.116564,0.863278
1,極洋,1995/01,1301.0,0.674847,0.223488,0.728309,0.717791,0.390009,0.699387,0.826468,...,0.500438,0.914987,0.780894,0.89220,0.803681,0.307625,0.044698,0.953550,0.116564,0.861525
2,極洋,1995/02,1301.0,0.433830,0.223488,0.687117,0.705521,0.393514,0.821209,0.827344,...,0.500438,0.912358,0.788782,0.89220,0.803681,0.441718,0.047765,0.952673,0.115688,0.906223
3,極洋,1995/03,1301.0,0.523225,0.236635,0.706398,0.725679,0.401402,0.868536,0.829097,...,0.500438,0.913234,0.800613,0.89220,0.803681,0.397897,0.049956,0.951797,0.116564,0.249781
4,極洋,1995/04,1301.0,0.799299,0.239264,0.723926,0.747590,0.403155,0.892200,0.829097,...,0.500438,0.914110,0.799737,0.89220,0.803681,0.548642,0.051271,0.951797,0.119194,0.091148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370820,ベルーナ,2021/08,9997.0,0.028922,0.551271,0.874671,0.086766,0.045574,0.601227,0.941280,...,0.292726,0.604733,0.213409,0.61525,0.673970,0.048203,0.488168,0.769500,0.470640,0.120947
370821,ベルーナ,2021/09,9997.0,0.672217,0.554777,0.843996,0.090272,0.045574,0.856266,0.941280,...,0.292726,0.604733,0.195004,0.61525,0.673970,0.172656,0.462752,0.769500,0.470640,0.625767
370822,ベルーナ,2021/10,9997.0,0.175285,0.550394,0.813322,0.092025,0.045574,0.393514,0.941280,...,0.292726,0.603856,0.188431,0.61525,0.673970,0.187555,0.505697,0.769500,0.470640,0.563541
370823,ベルーナ,2021/11,9997.0,0.066608,0.539001,0.691499,0.093777,0.044698,0.528484,0.941280,...,0.292726,0.601227,0.170026,0.61525,0.673094,0.042945,0.567046,0.769500,0.469763,0.041192


## 3.2 ランク正規化(0,1)の範囲を(-1,1)へ変更

In [7]:
data_4_7_3 = pd.DataFrame()
for g in range(data_4_7_0.iloc[:,3:].shape[1]):
    data_4_7_1 = data_4_7_0.iloc[:,3+g]
    data_4_7_2 = ((data_4_7_1-data_4_7_1.min())/(data_4_7_1.max()-data_4_7_1.min()))*(1-(-1))+(-1)
    data_4_7_3 = pd.concat([data_4_7_3, data_4_7_2], axis = 1)
#data_4_7_4 = pd.concat([data_4_7_0.iloc[:,:3], data_4_7_3], axis=1)
#display(data_4_7_4)
data_4_7 = pd.concat([data_4_7_0.iloc[:,:3], data_4_7_3], axis=1)

ones_1 = np.ones(int(com_num_4)*int(day_num_4))
ones_2 = pd.Series(ones_1, name="ones")
data_4_7 = pd.concat([data_4_7_4, ones_2], axis=1)

## 3.3 ランク正規化後データの書き出し

In [8]:
data_4_7.to_pickle('./ranked_data.pickle') #ファイルの書き出し, 配当込み収益率もランク正規化したものであることに注意

In [9]:
print(os.path.getsize('./ranked_data.pickle')) #ファイルの大きさを表示

130047405


# 4. 分析対象となる形状に変換

## 4.1 右側ファクターネットワークの入力データ

In [10]:
#returns_data_1 = data_4_7['配当込み収益率(月次)'] #収益率データのみ抽出, ランク正規化済
returns_data_1 = data_4['配当込み収益率(月次)'] #収益率データのみ抽出, ランク正規化未
returns_data_2 = np.split(returns_data_1, com_num_4, 0)
returns_data_3 = pd.DataFrame(returns_data_2[0])
for num in range(int(com_num_4)):
    returns_data_3 = np.concatenate([returns_data_3,pd.DataFrame(returns_data_2[num])], 1)
returns_data_4 = pd.DataFrame(returns_data_3)
returns_data_4 = returns_data_4.drop(returns_data_4.columns[[0]], axis=1) 
x = returns_data_4.to_numpy()
x.shape

(325, 1141)

## 4.2 左側ベータネットワークの入力データ

In [11]:
data_4_8 = data_4_7.drop(data_4_7.columns[[0, 1, 2]], axis=1) #余分な列を削除
data_4_8 = data_4_8.drop('配当込み収益率(月次)', axis=1) #配当込み収益率データを削除
chara_num = data_4_8.shape[1]
sh = data_4_8.shape #行と列の数を取り出す
#shapeの列の数だけrangeで振り直してcolumnsに代入
data_4_8.columns = range(sh[1])
data_4_9 = np.split(data_4_8, com_num_4, 0)
np.shape(data_4_9)

(1141, 325, 40)

In [12]:
data_4_10 = data_4_9[0]
for num in range(1,int(com_num_4)):
    data_4_10 = np.concatenate([data_4_10, data_4_9[num]], 1)
data_4_11 = pd.DataFrame(data_4_10)
y = data_4_11.iloc[:,0:].to_numpy()
y.shape 

(325, 45640)

# 5. 形状変換後データの書き出し(pickleによる高速化)

In [13]:
returns_data_4.to_pickle('./inright_data.pickle') #ファイルの書き出し, 配当込み収益率はランク正規化をしていないデータであることに注意
data_4_11.to_pickle('./inleft_data.pickle') #ファイルの書き出し

In [14]:
print(os.path.getsize('./inright_data.pickle'), os.path.getsize('./inleft_data.pickle')) #ファイルの大きさを表示

2967171 118664571
